Checkpoints sugeridos:

1) Coleta de dados via web scrapping. Bibliotecas como Selenium e BeautifulSoup podem ajudar.

2) Análise exploratória: Gráficos de dispersão entre nota padronizada dos dois rankings. São correlacionados? Existem outliers?

3) Modelo explicativo: Regressão linear múltipla entre os 4 itens do ranking do Legisla Brasil e a nota do Ranking dos Políticos, por exemplo. Os 4 itens do Legisla explicam qual variabilidade do Ranking dos Políticos?

4) Discussão: Por que são diferentes? Por que são iguais? Existem viéses políticos? São apartidários? Qual sua opinião que justifique os resultados?

### Observações para aplicar em novas versões do modelo:

- Dividir o dataset em dois, por nota ou por ideologia do partido.
- Fazer um teste de hipóteses 
- Transformar a variável categórica em numérica e adicionar ao modelo.

### Pontuações ao longo do caminho

 - Modelos de ML, como Regressão Linear, assumem que os dados são normalmente distribuidos. Se essa suposição é falsa o modelo falha em dar predições precisas.

# Machine Learning

In [1]:
# bibliotecas
import pandas as pd #processamento de dados
import numpy as np #algebra linear
import matplotlib.pyplot as plt # Matlab-Style
from pandas.plotting import scatter_matrix 
%matplotlib inline
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*arg, **kwargs):
    pass
warnings.warm = ignore_warn #ignorar avisos chatos (do sklearn e seaborn)

from scipy import stats
from scipy.stats import norm, skew #para analises estatítiscas

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # limita saídas de tipos floats 
                                                                    # a 3 casas decimais
from subprocess import check_output
print(check_output(["ls"]).decode("utf8")) #checar os arquivos disponiveis no diretório


# from sklearn import model_selection
# from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV

Analise_dos_dados.ipynb
BOLOGNESI_RIBEIRO_CODATO_nova+classificacao+ideologica+VERSAO+pre+print+jun2021.pdf
Busca automatica - webscraping.ipynb
MachineLearning.ipynb
Output.txt
README.md
Rank_pol_V3.ipynb
Ranking_Politicos.ipynb
Ranking_pol_site2.ipynb
Selenium-Airbnb.ipynb
chromedriver.exe
click_cada_dep.ipynb
df_after_AD.csv
df_merge.csv
df_site2.csv
rank_legislabr_completo.csv
rank_legislabr_describe.csv
rank_legislabr_notas.csv



In [14]:
# Carregando os dados
dados = pd.read_csv('df_after_AD.csv', dtype={'ideologia':'category'})

In [15]:
dados.head(1)

,Unnamed: 0,nome,estado,partido,ideologia,n_prod_leg,n_fisc,n_mobi,n_alin_partd,avg_site1,nota_site2
0,0,abou anni,São Paulo,UNIÃO,direita,3.800,0.900,2.400,6.400,2.853,6.550


In [16]:
# Visualizando as dimensões
dados = dados.drop('Unnamed: 0', axis=1)
dados.shape

(488, 10)

In [18]:
dados.head(2)

,nome,estado,partido,ideologia,n_prod_leg,n_fisc,n_mobi,n_alin_partd,avg_site1,nota_site2
0,abou anni,São Paulo,UNIÃO,direita,3.800,0.900,2.400,6.400,2.853,6.550
1,acacio favacho,Amapá,MDB,centro,2.600,0.200,2.100,6.400,2.029,6.120


**nota_site2** é a variável que desejamos predizer, no caso desse problema é a variável que queremos comparar com as variáveis independentes para saber se elas explicam a variabilidade da variável dependente.

## Treinando o modelo somente com as notas (Modelo M1)

O primeiro modelo foi realizado considerando somente as notas (PRODUÇÃO LEGISLATIVA, FISCALIZAÇÃO, MOBILIZAÇÃO e ALINHAMENTO PARTIDÁRIO) oferecidas pelo Site 1 que foram usadas como variáveis preditoras para tentar explicar a nota do site 2 (politicos.org.br), utilizada como variável alvo.

In [31]:
# dividindo o dataset
treino_M1, teste_M1 = train_test_split(dados, 
                                test_size=0.3,
                                random_state=42
                               )
treino_M1.shape, teste_M1.shape

((341, 10), (147, 10))

In [32]:
# Variável target
y_M1 = treino_M1.nota_site2.copy()

# Features preditoras
X_M1 = treino_M1.drop(['nota_site2', 'nome', 'ideologia', 'estado', 'partido'], axis=1).copy()

print(f"Número de linhas {X_M1.shape[0]}")
X_M1.head()

Número de linhas 341


,n_prod_leg,n_fisc,n_mobi,n_alin_partd,avg_site1
3,6.400,2.900,7.300,10.000,5.741
417,1.900,0.000,1.500,6.400,1.535
173,6.700,3.100,6.100,0.900,5.194
60,2.000,0.700,1.800,6.400,1.841
110,3.600,0.900,4.800,8.200,3.288


In [33]:
#dividindo os dados de treino em teste e treino com uma amostra estratificada
X_train_M1, X_val_M1, y_train_M1, y_val_M1 = train_test_split(
    X_M1, y_M1, 
    test_size=0.3,
    random_state=42
)

X_train_M1.shape, y_train_M1.shape, X_val_M1.shape, y_val_M1.shape

((238, 5), (238,), (103, 5), (103,))

In [37]:
# Treino do modelo M1
modelo_M1 = LinearRegression()
modelo_M1.fit(X_train_M1, y_train_M1)

LinearRegression()

In [38]:
modelo_M1.score(X_val_M1, y_val_M1)

0.2731510508907976

In [39]:
# Cross Validation
cv_scores = cross_val_score(modelo_M1, X_val_M1, y_val_M1,
                           cv=5,
                           scoring='r2'
                           )
print(f"Acurácia no Cross-validation: {cv_scores.mean()}")

Acurácia no Cross-validation: 0.31466415824726984


##### Avaliação do modelo M1

Aplicando um modelo de regressão linear e utilizando apenas as notas do site 1 (Legisla Brasil), temos um modelo **ruim**. O coeficiente de determinação R² é baixo, próximo aos 31%. E significa que o modelo linear explica 31% da variância da variável dependente a partir do regressores (variáveis independentes) incluídas neste modelo linear.

### Treinando o Modelo M2, com a variável categórica "ideologia"

Porque utilizar essa variável?
    Como foi visto na análise dos dados há uma separação das notas de acordo com a ideologia politica do partido ao qual o candidato faz parte.

In [41]:
colunas_num = ['n_prod_leg', 'n_fisc', 'n_mobi', 'n_alin_partd', 'avg_site1', 'nota_site2']
colunas_cat = ['ideologia']

### Dividindo o dataset 

In [42]:
treino_M2, teste_M2 = train_test_split(dados, 
                                test_size=0.3,
                                random_state=42
                               )
treino_M2.shape, teste_M2.shape

((341, 10), (147, 10))

In [127]:
y_test = teste_M2.nota_site2.copy()
X_test = teste_M2.drop(['nota_site2', 'nome', 'estado', 'partido'], axis=1).copy()

In [43]:
# Variável target
y_M2 = treino_M2.nota_site2.copy()

# Features preditoras
X_M2= treino_M2.drop(['nota_site2', 'nome', 'estado', 'partido'], axis=1).copy()

print(f"Número de linhas {X_M2.shape[0]}")
X_M2.head()

Número de linhas 341


,ideologia,n_prod_leg,n_fisc,n_mobi,n_alin_partd,avg_site1
3,direita,6.400,2.900,7.300,10.000,5.741
417,esquerda,1.900,0.000,1.500,6.400,1.535
173,direita,6.700,3.100,6.100,0.900,5.194
60,centro,2.000,0.700,1.800,6.400,1.841
110,direita,3.600,0.900,4.800,8.200,3.288


In [45]:
#dividindo os dados de treino em teste e treino com uma amostra estratificada
X_train_M2, X_val_M2, y_train_M2, y_val_M2 = train_test_split(
    X_M2, y_M2, 
    test_size=0.3,
    stratify=X["ideologia"],
    random_state=42
)

In [46]:
X_train_M2.shape, y_train_M2.shape, X_val_M2.shape, y_val_M2.shape

((238, 6), (238,), (103, 6), (103,))

In [47]:
# verificando a estratificação entre amostra e população
X_train_M2["ideologia"].value_counts()/len(X_train_M2)

direita    0.500
centro     0.265
esquerda   0.235
Name: ideologia, dtype: float64

In [48]:
dados["ideologia"].value_counts()/len(dados)

direita    0.490
centro     0.275
esquerda   0.236
Name: ideologia, dtype: float64

## Pipeline

Como temos uma base de dados pequena, vamos usar validação cruzada pra garantir medidas precisas da qualidade do modelo.

In [111]:
t = [('str_categoricas', OneHotEncoder(handle_unknown='ignore'), colunas_cat)]

preprocessor = ColumnTransformer(transformers=t, remainder='passthrough')

In [112]:
pipe_transformer = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('lr',LinearRegression())])

In [113]:
pipe_transformer.steps

[('preprocessor',
  ColumnTransformer(remainder='passthrough',
                    transformers=[('str_categoricas',
                                   OneHotEncoder(handle_unknown='ignore'),
                                   ['ideologia'])])),
 ('lr', LinearRegression())]

In [114]:
pipe_transformer.fit(X_train_M2, y_train_M2)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('str_categoricas',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['ideologia'])])),
                ('lr', LinearRegression())])

In [115]:
pipe_transformer.score(X_val_M2, y_val_M2)

0.6511458393732259

### Usando o cross-validation

In [116]:
cv_scores = cross_val_score(pipe_transformer, X_val_M2, y_val_M2,
                           cv=5,
                           scoring='r2'
                           )
print(f"Acurácia no Cross-validation: {cv_scores.mean()}")

Acurácia no Cross-validation: 0.5688530363123041


Com a adição da nova feature o modelo conseguiu uma performance melhor que o anterior, na casa dos **56,8.**

In [121]:
coeff_df = pd.DataFrame(pipe_transformer.steps[1][1].coef_,columns=['Coefficient'])
coeff_df

,Coefficient
0,316470253790.914
1,316470253791.146
2,316470253787.907
3,-5727223644362.283
4,-3579514777726.534
5,-2147708866635.918
6,-715902955545.364
7,12170350244270.025


In [117]:
pipe_transformer[:-1].get_feature_names_out()

array(['str_categoricas__ideologia_centro',
       'str_categoricas__ideologia_direita',
       'str_categoricas__ideologia_esquerda', 'remainder__n_prod_leg',
       'remainder__n_fisc', 'remainder__n_mobi',
       'remainder__n_alin_partd', 'remainder__avg_site1'], dtype=object)

In [120]:
pipe_transformer.steps[1][1].coef_

array([ 3.16470254e+11,  3.16470254e+11,  3.16470254e+11, -5.72722364e+12,
       -3.57951478e+12, -2.14770887e+12, -7.15902956e+11,  1.21703502e+13])